In [ ]:
from database_handler import create_connection, return_data_as_df
import pandas as pd
from lookups import InputTypes
def standings_points_rolling(db_session):

    dict_teams_season = {}
    for season in range(2018,2024):
        query = """WITH wind AS (
        SELECT
            CASE
            WHEN home_team_goals > away_team_goals THEN home_team_name
            WHEN home_team_goals < away_team_goals THEN away_team_name
            ELSE 'Draw'
            END AS winner,
            *
        FROM premier_league.fact_game_results
        ORDER BY match_id
        ),
        new_wind AS (
        SELECT
            CASE
            WHEN SUBSTRING(CAST(match_id AS TEXT), 0, 3) = '38' THEN 2018
            WHEN SUBSTRING(CAST(match_id AS TEXT), 0, 3) = '46' THEN 2019
            WHEN SUBSTRING(CAST(match_id AS TEXT), 1, 2) IN ('58', '59') THEN 2020
            WHEN SUBSTRING(CAST(match_id AS TEXT), 0, 3) = '66' THEN 2021
            WHEN SUBSTRING(CAST(match_id AS TEXT), 0, 3) IN ('74', '75') THEN 2022
            WHEN SUBSTRING(CAST(match_id AS TEXT), 0, 3) = '93' THEN 2023
            END AS season,
            *
        FROM wind)

        SELECT * FROM new_wind
        """
        concatenated_df = None
        df = return_data_as_df(file_executor=query, input_type=InputTypes.SQL, db_session=db_session)
        df = df[df['season'] == season] #changes in loop
        list_of_teams = df['home_team_name'].unique().tolist()
        dict_teams_season[season] = list_of_teams
        for team in list_of_teams:
            query = f"""WITH wind AS (
                    SELECT
                        CASE
                        WHEN home_team_goals > away_team_goals THEN home_team_name
                        WHEN home_team_goals < away_team_goals THEN away_team_name
                        ELSE 'Draw'
                        END AS winner,
                        *
                    FROM premier_league.fact_game_results
                    ORDER BY match_id
                    ),
                    new_wind AS (
                    SELECT
                        CASE
                        WHEN SUBSTRING(CAST(match_id AS TEXT), 0, 3) = '38' THEN 2018
                        WHEN SUBSTRING(CAST(match_id AS TEXT), 0, 3) = '46' THEN 2019
                        WHEN SUBSTRING(CAST(match_id AS TEXT), 1, 2) IN ('58', '59') THEN 2020
                        WHEN SUBSTRING(CAST(match_id AS TEXT), 0, 3) = '66' THEN 2021
                        WHEN SUBSTRING(CAST(match_id AS TEXT), 0, 3) IN ('74', '75') THEN 2022
                        WHEN SUBSTRING(CAST(match_id AS TEXT), 0, 3) = '93' THEN 2023
                        END AS season,
                        *
                    FROM wind
                    WHERE home_team_name = '{team}' OR away_team_name = '{team}'
                    ),
                    points_selected AS (
                    SELECT
                        CASE
                        WHEN winner = '{team}' THEN 3
                        WHEN winner = 'Draw' THEN 1
                        ELSE 0
                        END AS points_selected_team,
                        *
                    FROM new_wind
                    )

                    SELECT '{team}' AS Selected_team,
                    SUM(points_selected_team) OVER (PARTITION BY season ORDER BY match_id) AS running_points_total,
                    *
                    FROM points_selected;"""
            
            df = return_data_as_df(file_executor=query, input_type=InputTypes.SQL, db_session=db_session)
            concatenated_df = pd.concat([concatenated_df,df], axis=0, ignore_index=True)
        return concatenated_df

In [ ]:
db_session = create_connection()
standings_points_rolling(db_session)

In [1]:
from database_handler import create_connection
from prehook import insert_points_standings_into_fact

db_session = create_connection()
insert_points_standings_into_fact(db_session)

c:\Users\96171\Desktop\SE_Factory\DataAnalysis_Jupiter\database_handler.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_executor)


Done creating stg table of: rolling_points_standing
Inserting into: rolling_points_standing
